**Preppin' Data 2022 Week 7**

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#Importing Data from people
people=pd.read_excel('C:/Users/yash/Downloads/PeopleData.xlsx',sheet_name='People')
leaders=pd.read_excel('C:/Users/yash/Downloads/PeopleData.xlsx',sheet_name='Leaders')
location=pd.read_excel('C:/Users/yash/Downloads/PeopleData.xlsx',sheet_name='Location')
goals=pd.read_excel('C:/Users/yash/Downloads/PeopleData.xlsx',sheet_name='Goals')
dates=pd.read_excel('C:/Users/yash/Downloads/PeopleData.xlsx',sheet_name='Date Dim')

In [3]:
data=pd.ExcelFile('MetricData2021.xlsx')

In [4]:
col_names={'Offered':'Calls Offered','Not Answered':'Calls Not Answered','Answered':'Calls Answered'}

In [5]:
#Importing metrics file and concatenating sheets
metrics=pd.DataFrame(data=None)
for s in data.sheet_names:
    d=pd.DataFrame(data=None)
    d=pd.read_excel(data,sheet_name=s)
    d=d.rename(col_names,axis=1)
    d=d.assign(Month=s)
    metrics=metrics.append(d)

In [6]:
metrics.shape

(1516, 8)

In [7]:
#Merging People & leaders
ag_le=people.merge(leaders,how='left',left_on='Leader 1',right_on='id',suffixes=('_agent','_leader'))

In [8]:
#Merging People & leaders
final=ag_le.merge(location,how='left')

In [9]:
final=final.drop(['id_leader','Location ID'],axis=1)

In [10]:
#adding agent and leader name column
final['Agent Name']=final['last_name_agent']+", "+final['first_name_agent']
final['Leader Name']=final['last_name_leader']+", "+final['first_name_leader']

In [11]:
final.drop(['first_name_agent','last_name_agent','first_name_leader','last_name_leader'],axis=1,inplace=True)

In [12]:
final.head()

,id_agent,Leader 1,Location,Agent Name,Leader Name
0,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie"
1,8,3,Margaree,"Jobbings, Tandi","Befroy, Yorke"
2,10,1,Margaree,"Beaver, Leanora","Howroyd, Kylie"
3,11,1,Margaree,"Cranke, Zabrina","Howroyd, Kylie"
4,12,1,Margaree,"Matysiak, Berny","Howroyd, Kylie"


In [13]:
#Filter year to 2021
dates=dates[dates['Month Start Date'].dt.year<2022]

In [14]:
#join dates dataframe to people location and leaders data
final['key'] = 1
dates['key'] = 1
  
# to obtain the cross join we will merge 
# on the key and drop it.
final_1= pd.merge(final, dates, on ='key').drop("key", 1)

In [16]:
#adding start date to metrics data
metrics['Month']=metrics['Month'].apply(lambda x: dt.datetime(2021,int(dt.datetime.strptime(x, '%b').strftime("%m")),1))

In [18]:
final_1.head()

,id_agent,Leader 1,Location,Agent Name,Leader Name,Month Start Date
0,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-01-01
1,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-02-01
2,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-03-01
3,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-04-01
4,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-05-01


In [21]:
#joining metrics data to people data
df=final_1.merge(metrics,how='left',left_on=['id_agent','Month Start Date'],right_on=['AgentID','Month'])
df=df.drop(['AgentID','Month'],axis=1)

In [23]:
#create a calculation for the percent of offered that weren't answered (for each agent, each month)
df['Not Answered Rate']=(df['Calls Not Answered']/df['Calls Offered']).round(3)

In [25]:
#create a calculation for the average duration by agent (for each agent, each month)
df['Agent Avg Duration']=(df['Total Duration']/df['Calls Offered']).round().astype('Int64')

In [66]:
goals['goal']=goals['Goals'].str.extract('([A-Za-z]+\s?\w+\s?\w+)')
goals['Cut-off']=goals['Goals'].str[-1:].astype('Float64')

In [67]:
goals.head()

,Goals,goal,Cut-off
0,Not Answered Percent < 5,Not Answered Percent,5.0
1,Sentiment Score >= 0,Sentiment Score,0.0


In [70]:
df.head()

,id_agent,Leader 1,Location,Agent Name,Leader Name,Month Start Date,Calls Offered,Calls Not Answered,Calls Answered,Total Duration,Sentiment,Transfers,Not Answered Rate,Agent Avg Duration
0,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-01-01,445.0,2.0,443.0,7565.0,-53.0,NaN,0.004,17
1,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-02-01,606.0,16.0,590.0,4848.0,97.0,NaN,0.026,8
2,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-03-01,413.0,75.0,338.0,2478.0,23.0,NaN,0.182,6
3,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-04-01,760.0,12.0,748.0,6080.0,21.0,NaN,0.016,8
4,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-05-01,486.0,22.0,464.0,1458.0,-33.0,23.0,0.045,3


In [71]:
df['Not Answered Percent']=float(goals[goals['goal']=="Not Answered Percent"]['Cut-off'])
df['Sentiment Score >= 0']=float(goals[goals['goal']=="Sentiment Score"]['Cut-off'])

In [72]:
df.head()

,id_agent,Leader 1,Location,Agent Name,Leader Name,Month Start Date,Calls Offered,Calls Not Answered,Calls Answered,Total Duration,Sentiment,Transfers,Not Answered Rate,Agent Avg Duration,Not Answered Percent,Sentiment Score >= 0
0,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-01-01,445.0,2.0,443.0,7565.0,-53.0,NaN,0.004,17,5.0,0.0
1,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-02-01,606.0,16.0,590.0,4848.0,97.0,NaN,0.026,8,5.0,0.0
2,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-03-01,413.0,75.0,338.0,2478.0,23.0,NaN,0.182,6,5.0,0.0
3,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-04-01,760.0,12.0,748.0,6080.0,21.0,NaN,0.016,8,5.0,0.0
4,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-05-01,486.0,22.0,464.0,1458.0,-33.0,23.0,0.045,3,5.0,0.0


In [77]:
df['Met Not Answered Rate']=np.where((df['Not Answered Rate']<(df['Not Answered Percent']/100)),True,False)
df['Met Sentiment Goal']=np.where((df['Sentiment']>(df['Sentiment Score >= 0'])),True,False)

In [78]:
df.head()

,id_agent,Leader 1,Location,Agent Name,Leader Name,Month Start Date,Calls Offered,Calls Not Answered,Calls Answered,Total Duration,Sentiment,Transfers,Not Answered Rate,Agent Avg Duration,Not Answered Percent,Sentiment Score >= 0,Met Not Answered Rate,Met Sentiment Goal
0,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-01-01,445.0,2.0,443.0,7565.0,-53.0,NaN,0.004,17,5.0,0.0,True,False
1,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-02-01,606.0,16.0,590.0,4848.0,97.0,NaN,0.026,8,5.0,0.0,True,True
2,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-03-01,413.0,75.0,338.0,2478.0,23.0,NaN,0.182,6,5.0,0.0,False,True
3,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-04-01,760.0,12.0,748.0,6080.0,21.0,NaN,0.016,8,5.0,0.0,True,True
4,4,1,Margaree,"Garnam, Fleur","Howroyd, Kylie",2021-05-01,486.0,22.0,464.0,1458.0,-33.0,23.0,0.045,3,5.0,0.0,True,False


In [79]:
df.shape

(1620, 18)

In [83]:
df.to_csv('Preppin Data Week 7 Output-YS.csv',index=False)